#### Dependecies

scipy = 0.17.0
gensim = 1.0.1
pandas = 0.19.2
numpy = 1.11.2

#### I used the following code in Pyspark to generate and save on disk the csv file below

select_noteevents2 = spark.sql("""
SELECT text
FROM noteevents2
""")

select_noteevents2.coalesce(1).write.csv("/Volumes/EXTERNAL1/MIMICIII/select_noteevents2-v3", header=True)

select_noteevents3 = spark.sql("""
SELECT subject_id, hadm_id
FROM noteevents2
""")

select_noteevents3.coalesce(1).write.csv("/Volumes/EXTERNAL1/MIMICIII/select_noteevents2-v4", header=True)

#### then I proceed to open a new notebook to run pandas and gensim 

In [2]:
import pandas as pd
df = pd.read_csv('/Volumes/EXTERNAL1/MIMICIII/select_noteevents2-v3/part-00000-ac3edd68-1571-4e16-a2c6-ada909da0eea.csv',sep='delimiter')
df.head(5)

/Users/cesarosorio/anaconda/envs/BDFH_project/lib/python2.7/site-packages/ipykernel/__main__.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  from ipykernel import kernelapp as app


,text
0,"""Admission Date: [**2183-9-25**] Discha..."
1,"""Admission Date: [**2184-1-16**] Discha..."
2,"""Admission Date: [**2103-4-11**] ..."
3,"""Admission Date: [**2103-10-7**] Discha..."
4,"""\""Admission Date: [**2131-4-2**] ..."


In [2]:
df_ids = pd.read_csv('/Volumes/EXTERNAL1/MIMICIII/select_noteevents2-v4/part-00000-9b6dc461-8bf9-4ae4-b626-2156a12031ab.csv')
df_ids.head(5)

,subject_id,hadm_id
0,20007,188442
1,20007,193793
2,59883,118446
3,17043,157985
4,7019,189488


In [ ]:
# Cleanning the data
import re
def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    text = re.sub('[\W]+', ' ', text.lower()) 
    return text

df['text3'] = df['text'].apply(preprocessor)

In [ ]:
# Create tokens
token_review=[]
for i in range(df['text3'].shape[0]):
    review = df['text3'][i]
    token_review.append([i for i in review.split()])

len(token_review)

### Apply Word2Vec

In [26]:
from gensim.models import Word2Vec
#import gensim.models.Word2Vec
from gensim import utils
from time import time

# assumptions: window is 5 words left and right, eliminate words than dont occur in
# more than 10 docs, use 4 workers for a quadcore machine. Size is the size of vector
# negative=5 implies negative sampling and makes doc2vec faster to train
#model = Doc2Vec(sentence, size=100, window=5, workers=4, min_count=5)


import random

size = 100

#instantiate our  model
model_w2v = Word2Vec(min_count=10, window=5, size=size, sample=1e-3, negative=5, workers=4)

#build vocab over all reviews
model_w2v.build_vocab(token_review)

#We pass through the data set multiple times, shuffling the training reviews each time to improve accuracy.
Idx=list(range(len(token_review)))

t0 = time()
for epoch in range(5):
     random.shuffle(Idx)
     perm_sentences = [token_review[i] for i in Idx]
     model_w2v.train(perm_sentences)
     print(epoch)
    
elapsed=time() - t0
print("Time taken for Word2vec training: ", elapsed, "seconds.")

0
1
2
3
4
('Time taken for Word2vec training: ', 766.5733489990234, 'seconds.')


In [3]:
# saves the word2vec model to be used later.
#model_w2v.save('./model_word2vec')

# open a saved word2vec model 
#import gensim
#model_w2v=gensim.models.Word2Vec.load('./model_word2vec')


#model_w2v.wv.save_word2vec_format('./model_word2vec.txt', binary=False)



#### Run  if you are using Glove type format


In [6]:
# Run this cell if you are using Glove type format
def loadGloveModel(gloveFile):
    print("Loading Glove Model")
    f = open(gloveFile,'r')
    model = {}
    for line in f:
        splitLine = line.split()
        word = splitLine[0]
        embedding = [float(val) for val in splitLine[1:]]
        model[word] = embedding
    print("Done.",len(model)," words loaded!")
    return model

model_w2v=loadGloveModel("./model_word2vec.txt")

Loading Glove Model
('Done.', 33837, ' words loaded!')


In [1]:
# Code I used on PySpark to create tables below

#df_labels=get_id_to_texticd9("hadm_id", 10)[0]
#df_labels.registerTempTable("labels")
#select_text = spark.sql(""" SELECT text FROM labels """)
#select_labels = spark.sql(""" SELECT id, `4019`, `2724`,`25000`,`4280`,`41401`,`53081`,`51881`,`42731`,`5849`,`5990` FROM labels """)
#select_text.coalesce(1).write.csv("/Volumes/EXTERNAL1/MIMICIII/select_text", header=True)
#select_labels.coalesce(1).write.csv("/Volumes/EXTERNAL1/MIMICIII/select_labels", header=True)

#import data
import pandas as pd
df = pd.read_csv('/Volumes/EXTERNAL1/MIMICIII/select_text/part-00000-06489b47-a266-4613-b2f7-aa7031961e44.csv', sep='delimiter')
df.head(5)

/Users/cesarosorio/anaconda/envs/BDFH_project/lib/python2.7/site-packages/ipykernel/__main__.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  app.launch_new_instance()


,text
0,"""\""Admission Date: [**2118-12-14**] ..."
1,"""Admission Date: [**2137-8-31**] ..."
2,"""Admission Date: [**2174-1-6**] ..."
3,"""Admission Date: [**2102-6-9**] ..."
4,"""Admission Date: [**2164-7-2**] Dischar..."


In [3]:
#import data
import pandas as pd
df_labels = pd.read_csv('/Volumes/EXTERNAL1/MIMICIII/select_labels/part-00000-8c86e2d9-b1a9-493e-815e-372ed1d8cd84.csv')
df_labels.head(5)

,id,4019,2724,25000,4280,41401,53081,51881,42731,5849,5990
0,117760,0,0,0,0,0,1,1,0,0,0
1,129030,1,1,0,0,0,1,0,0,0,0
2,172040,0,0,0,0,1,0,0,0,1,0
3,156170,0,0,1,1,0,0,0,1,1,0
4,199180,0,0,1,1,1,0,0,0,0,0


In [4]:
# Cleanning the data
import re

def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    text = re.sub('[\W]+', ' ', text.lower()) 
    text = re.sub(" \d+", " ", text)
    #text = gensim.parsing.preprocessing.remove_stopwords(text)
    return text


df['text3'] = df['text'].apply(preprocessor)

# Create tokens
token_review=[]
for i in range(df['text3'].shape[0]):
    review = df['text3'][i]
    token_review.append([i for i in review.split()])

len(token_review)

40562

In [7]:
import numpy as np  # Make sure that numpy is imported
from nltk.corpus import stopwords

keys_updated = [word for word in model_w2v.keys() if word not in stopwords.words('english')]
index2word_set=set(keys_updated)

def makeFeatureVec(words, model, num_features):
    # Function to average all of the word vectors in a given
    # paragraph
    #
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,),dtype="float32")
    #
    nwords = 0.
    # 
    # Index2word is a list that contains the names of the words in 
    # the model's vocabulary. Convert it to a set, for speed 
    #index2word_set = set(model.wv.index2word) #activate if using gensim

    # activate if uploaded text version
    #index2word_set=set(keys_updated)
    
    
    #
    # Loop over each word in the review and, if it is in the model's
    # vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set: 
            nwords = nwords + 1.
            featureVec = np.add(featureVec,model[word])
    # 
    # Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec,nwords)
    return featureVec


In [8]:
makeFeatureVec(token_review[0], model_w2v, 100)

array([ 0.44611126,  0.4078805 , -0.60862143, -0.10496631,  0.58714236,
        0.32758738, -0.35371386,  0.14757633, -0.15158055, -0.61058004,
        0.66827482, -0.91480946, -0.64827127,  0.8269808 , -0.21633704,
       -0.19003792, -0.4626302 ,  0.86939905, -0.02538301,  1.24280317,
        0.70031001, -0.32402792, -0.37870119, -0.62153993, -0.8197432 ,
        0.56903719,  0.88334291, -0.44279147,  0.86053797, -0.31016905,
        0.19820469,  0.33730923,  0.07074093,  0.50636062,  0.20623061,
       -0.59894437,  1.1693294 , -0.5786109 ,  0.00204187, -0.18991234,
       -0.6600972 , -0.15451911, -0.20741686,  1.60779081, -0.41607993,
       -0.33875471, -0.52661974,  0.18455352,  0.03641037, -1.46472083,
        0.6435804 , -0.50940622, -0.74418067, -0.31860323,  0.91815633,
        0.41322664, -0.78944104, -0.95176658, -1.0885331 , -0.41755329,
        0.86356699,  0.24149995, -0.05949575, -0.33603666, -0.96479437,
       -0.10660117,  0.07605942,  0.54283915,  0.03647094,  0.64

In [9]:
def getAvgFeatureVecs(reviews, model, num_features):
    # Given a set of reviews (each one a list of words), calculate 
    # the average feature vector for each one and return a 2D numpy array 
    # 
    # Initialize a counter
    counter = 0
    # 
    # Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    # 
    # Loop through the reviews
    for review in reviews:
       #
       # Print a status message every 10000th review
       if counter%10000 == 0:
           print "Review %d of %d" % (counter, len(reviews))
       # 
       # Call the function (defined above) that makes average feature vectors
       reviewFeatureVecs[counter] = makeFeatureVec(review, model,num_features)
       #
       # Increment the counter
       counter = counter + 1
    return reviewFeatureVecs

In [10]:
from time import time
t0 = time()
final_w2v=getAvgFeatureVecs(token_review, model_w2v, num_features=100)
elapsed=time() - t0
print("Time taken for Word2vec avg vector per note calculation: ", elapsed, "seconds.")

Review 0 of 40562
Review 10000 of 40562
Review 20000 of 40562
Review 30000 of 40562
Review 40000 of 40562
('Time taken for Word2vec avg vector per note calculation: ', 180.15606999397278, 'seconds.')


#### Create train set and test set to use Machine Learning model


In [99]:
# Create train set and test set to use Machine Learning model
import random

final_w2v_df=pd.DataFrame(data=final_w2v)  
data_final=pd.concat([df_labels,final_w2v_df], axis=1)

idx=list(range(len(token_review)))
random.seed(1234)
random.shuffle(idx)
idx_train= idx[0:int(len(data_final)*0.50)]
idx_val= idx[int(len(data_final)*0.50):int(len(data_final)*0.75)]
idx_test= idx[int(len(data_final)*0.75):len(data_final)]

train_set = data_final.iloc[idx_train]
val_set = data_final.iloc[idx_val]
test_set =data_final.iloc[idx_test]

print(train_set.shape, val_set.shape  ,test_set.shape)

((20281, 111), (10140, 111), (10141, 111))


#### Top 10 notes similar to note '0' or first note in dataset (to compare results to Doc2vec)


In [85]:
# Top 10 notes similar to note '0' or first note in dataset (to compare results to Doc2vec)

import scipy
from scipy import spatial

cosine_results=[]
for i in range(0,final_w2v.shape[0]):
    result=0
    result = 1 - spatial.distance.cosine(final_w2v[0], final_w2v[i])
    cosine_results.append(float("{0:.3f}".format(result)))
    
list_index=sorted(range(len(cosine_results)), key=lambda i: cosine_results[i] , reverse=True)[:11]
for i in list_index:
    print i, cosine_results[i]


0 1.0
12916 0.878
29259 0.875
34475 0.861
35623 0.856
38291 0.855
40477 0.846
35309 0.845
16038 0.839
4889 0.837
36676 0.837


### Try TFIDF

In [86]:
# Transform the dataframe to a sparse Tdidf matrix
#from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

# Now we create the sparse matrix of tfidf values
tfidf = TfidfVectorizer(input='content',ngram_range=(1, 1),stop_words='english', min_df=10, max_df=0.8)
# I select to remove stopwords and minimun doc frequency =10 to delete very unusual words
# that only show up in less than 10 notes (out of 59k notes available) 

dtm = tfidf.fit_transform([c for c in df['text3']])
vocab = np.array(tfidf.get_feature_names())
dtm = dtm.toarray()  # convert to a regular array
vocab = np.array(vocab)
dtm.shape


(40562, 21988)

In [101]:
# Create train set and test set to use Machine Learning model
# you need to add the label
import random

dtm_df=pd.DataFrame(data=dtm)  
data_tfidf=pd.concat([df_labels,dtm_df], axis=1)

train_set_tfidf = data_tfidf.iloc[idx_train]
val_set_tfidf = data_tfidf.iloc[idx_val]
test_set_tfidf = data_tfidf.iloc[idx_test]

print(train_set_tfidf.shape, val_set_tfidf.shape  ,test_set_tfidf.shape)

((20281, 21999), (10140, 21999), (10141, 21999))
